In [0]:
import numpy as np
import csv
import pandas as pd
import os
import time
from tqdm import tqdm

In [0]:
def compute_metrics(df_true, df_pred):
    
    """Compute MRR metric
    Assumptions: Ensure we have top 10 results for all the questions for the code sanity
    Sort the results by cosine similarity and give the top 10
    """
    all_scores = {}
    MRR = 0
    
    
    #list of true and predicted
    list1= list(set(df_pred['qid'].to_list()))
    list2= list(set(df_true['qid'].to_list()))
    
    
    for qid in list1:
        if qid in list2:
            #print (qid)
            target_pid = list(df_true[df_true['qid']==qid]['pid'])
            candidate_pid = list(df_pred[df_pred['qid']==qid]['pid'])
            rank_pid = list(df_pred[df_pred['qid']==qid]['rank'])
            
            for i in range(0,MaxMRRRank):
                if candidate_pid[i] in target_pid:
                    MRR += 1/(rank_pid[i] + 1)
                    #print (MRR)
                

    
    
    #calculating the final MRR
    MRR = MRR/len(list1)
    all_scores['MRR @10'] = MRR
    all_scores['QueriesRanked'] = len(list1)
    return all_scores

#### MRR Type 1 Evaluation

In [0]:
MaxMRRRank = 4

In [0]:
#Input Dataframe
df_true = pd.DataFrame.from_dict([
    {'qid': 'q1', 'pid': 'doc2'},
    {'qid': 'q1', 'pid': 'doc3'},
    {'qid': 'q2', 'pid': 'doc7'},
])



#Predicted dataframe
df_pred = pd.DataFrame.from_dict([
    {'qid': 'q1', 'pid': 'doc1', 'rank': 1},
    {'qid': 'q1', 'pid': 'doc2', 'rank': 2},
    {'qid': 'q1', 'pid': 'doc3', 'rank': 3},
    {'qid': 'q1', 'pid': 'doc4', 'rank': 4},
    {'qid': 'q2', 'pid': 'doc4', 'rank': 1},
    {'qid': 'q2', 'pid': 'doc5', 'rank': 2},
    {'qid': 'q2', 'pid': 'doc6', 'rank': 3},
    {'qid': 'q2', 'pid': 'doc7', 'rank': 4},
])



In [5]:
df_true

,qid,pid
0,q1,doc2
1,q1,doc3
2,q2,doc7


In [6]:
df_pred

,qid,pid,rank
0,q1,doc1,1
1,q1,doc2,2
2,q1,doc3,3
3,q1,doc4,4
4,q2,doc4,1
5,q2,doc5,2
6,q2,doc6,3
7,q2,doc7,4


In [7]:
mrr_type1 = compute_metrics(df_true, df_pred)
mrr_type1

{'MRR @10': 0.39166666666666666, 'QueriesRanked': 2}

##### How it has come : 
(1/3 + 1/4 + 1/5)/2  = 0.39166666666

#### MRR-Type 2 Evaluation

In [8]:
MAX_RANK = 4

hits = pd.merge(df_true, df_pred,
    on=["qid", "pid"],
    how="left").fillna(MAX_RANK)

mrr_type2 = (1 / hits.groupby('qid')['rank'].min()).mean()
mrr_type2

0.375